# Imports

In [3]:
import json
import boto3

# Create a session

Remember to activate the secret keys in "Add-ons"

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [5]:
region = input("Type in your region: ")

Type in your region:  eu-west-1


In [6]:
try:
    session = boto3.Session(
        aws_access_key_id=user_secrets.get_secret("access_key"),
        aws_secret_access_key=user_secrets.get_secret("secret_access_key"),
        region_name=region
    )
except Exception as e:
    print("Error occurred while creating session: ", e)

In [7]:
try:
    personalize = session.client('personalize')
    personalize_runtime = session.client('personalize-runtime')
except Exception as e:
    print("Error creating client: ", e)

# Train Recommender

## Choose a recommender use cases

In [8]:
name = input('Type in desired name of recommender: ')

Type in desired name of recommender:  fast-bachelor-project-recommender


In [9]:
dataset_group_arn = input('Type in your dataset group name: ')

Type in your dataset group name:  arn:aws:personalize:eu-west-1:267355461125:dataset-group/fast-bachelor-project


In [10]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])

[{'name': 'aws-vod-because-you-watched-x',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-because-you-watched-x',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2023, 3, 22, 0, 22, 3, 123000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},
 {'name': 'aws-vod-more-like-x',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2023, 3, 22, 0, 22, 3, 123000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},
 {'name': 'aws-vod-most-popular',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-vod-most-popular',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2023, 3, 22, 0, 22, 3, 123000, tzinfo=tzlocal()),
  'domain': 'VIDEO_ON_DEMAND'},
 {'

In [11]:
# Creating a 'top picks for you' recommender. Change recipeArn if you want a different recommender.
try:
    create_recommender_response = personalize.create_recommender(
      name = name,
      recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
      datasetGroupArn = dataset_group_arn
    )
    print('Recommender created successfully')
except Exception as e:
    print(f'Error creating recommender: {e}')

Recommender created successfully


In [12]:
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print ('Recommender ARN:', recommender_top_picks_arn)

Recommender ARN: arn:aws:personalize:eu-west-1:267355461125:recommender/fast-bachelor-project-recommender


#### Now wait until the recomenders have finished creating and have status `ACTIVE` in AWS, and then the recommender is ready.